<a href="https://colab.research.google.com/github/BostonGene-education/MIPT/blob/main/2020/RNA-Seq/03.%20RNA-Seq%20tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://blastim.ru/wp-content/uploads/wpjobboard/company/49/company-logo/bostongene-logo.jpg" width=500 align="center">

# Практикум по анализу RNA-Seq

<b>Автор</b>: Сергей Исаев

## Загрузка прочтений

На лекции мы с вами говорили о базе данных NCBI SRA. Для того, чтобы загружать данные оттуда, необходим специальный пакет программ SRA Toolkit. Этот пакет плохо ставится на Google Colab, поэтому мы загрузим данные из другой большой базы &mdash; <a href="https://www.ebi.ac.uk/arrayexpress/" target="_blank">ArrayExpress</a> при помощи <code>wget</code>.

Из датасета с ID <a href="https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-9724/" target="_blank">E-MTAB-9724</a> загрузим несколько прочтений экспериментов RNA-Seq клеточных линий LUAD (аденокарциномы лёгкого) &mdash; с нокаутом гена <i>STK11</i> и с диким типом. <b>Загрузка прочтений может занять много времени.</b>

In [ ]:
!mkdir raw_reads
!cd raw_reads && mkdir sample_7 sample_8 sample_9 sample_10 sample_11 sample_12

#Wild type
!cd raw_reads/sample_7 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/002/ERR4781422/ERR4781422_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/002/ERR4781422/ERR4781422_2.fastq.gz
!cd raw_reads/sample_8 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/003/ERR4781423/ERR4781423_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/003/ERR4781423/ERR4781423_2.fastq.gz
!cd raw_reads/sample_9 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/004/ERR4781424/ERR4781424_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/004/ERR4781424/ERR4781424_2.fastq.gz

#STK11-/-
!cd raw_reads/sample_10 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/005/ERR4781425/ERR4781425_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/005/ERR4781425/ERR4781425_2.fastq.gz
!cd raw_reads/sample_11 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/006/ERR4781426/ERR4781426_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/006/ERR4781426/ERR4781426_2.fastq.gz
!cd raw_reads/sample_12 && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/007/ERR4781427/ERR4781427_1.fastq.gz && \
    wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR478/007/ERR4781427/ERR4781427_2.fastq.gz

В папке <code>raw_reads</code> у нас есть подпапки, соответствующие анализируемым экспериментам. В них находятся по два файла &mdash; с прямыми и обратными прочтениями. Сначала нам необходимо установить программы <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/" target="_blank">FastQC</a> и <a href="https://multiqc.info/" target="_blank">MultiQC</a> и провести контроль качества прочтений. Первая программа оценит разнообразные метрики качества ридов каждого образца, вторая &mdash; соберёт все аутпуты FastQC в один красивый. <b>Контроль качества каждого файла занимает по несколько минут.</b>

In [ ]:
!sudo apt install fastqc
!mkdir  raw_reads/fastqc
!fastqc -o raw_reads/fastqc raw_reads/sample_*/*.fastq.gz

На основе получившихся файлов FastQC запускаем MultiQC.

In [ ]:
!pip install multiqc
!mkdir raw_reads/multiqc
!multiqc -o raw_reads/multiqc raw_reads/fastqc/

После этого можно загрузить .html-отчёт о качестве (сейчас он находится в папке <code>raw_reads/multiqc</code>) себе на компьютер и посмотреть, всё ли хорошо с прочтениями (копия отчёта лежит в папке <code>data</code> под названием <code>00. MultiQC report.html</code>. Рассмотрим каждый пункт отчёта:
1. Sequence Quality Histograms
2. Per Sequence Quality Scores
3. <b>Per Base Sequence Content</b> &mdash; проблемы тут связаны с особенностью подготовки библиотеки (использовались случайные праймеры)
4. Per Sequence GC Content
5. Per Base N Content
6. Sequence Length Distribution
7. <b>Sequence Duplication Levels</b> &mdash; в основе порогов данной метрики лежит предположение, что высокое количество дублирующихся прочтений говорит об артефактах ПЦР (это справедливо для, например, геномных библиотек, в которых встретить два одинаковых рида очень маловероятно). Однако для RNA-Seq-экспериментов это нормально, потому что какие-то гены могут встречаться очень часто из-за высокой копийности их РНК (<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/8%20Duplicate%20Sequences.html" target="_blank">источник</a>)
8. <b>Overrepresented sequences</b> &mdash; проблема вызвана аналогичными седьмому пункту проблемами (<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/9%20Overrepresented%20Sequences.html" target="_blank">источник</a>)
9. <b>Adapter Content</b> &mdash; мы видим, что у нас в большом числе прочтений содержатся технические адаптеры. Однако почти все современные алгоритмы для работы с RNA-Seq (в т.ч. и kallisto и STAR) автоматически прорабатывают эту проблему, поэтому мануального устранения не требуется (<a href="https://dnatech.genomecenter.ucdavis.edu/faqs/when-should-i-trim-my-illumina-reads-and-how-should-i-do-it/" target="_blank">про то, когда и зачем лучше обрезать адаптеры</a>)

Из этого всего можно сделать вывод, что прочтения у нас хорошие и с ними можно работать.

## Подсчёт экспрессий

Непосредственно оценка экспрессий будет производиться при помощи программы <a href="https://pachterlab.github.io/kallisto/" target="_blank">kallisto</a> (<a href="https://www.nature.com/articles/nbt.3519" target="_blank">Bray et al, 2016</a>). В отличие от других подходов, в kallisto отсутствует стадия выравнивания прочтений на референсный геном или транскриптом. Данный подход строит граф де Брёйна из k-меров транскриптома, после k-меры прочтений образуют пути в этом графе. Основываясь на этих путях, производится максимально правдоподобная оценка экспрессий различных транскриптов.

Подробнее про работу метода можно прочитать <a href="https://bioinfo.iric.ca/understanding-how-kallisto-works/#:~:text=The%20idea%20behind%20kallisto%20is,(not%20the%20whole%20genome)." target="_blank">здесь</a>. Первым шагом мы загрузим kallisto из git-репозитория и установим эту программу.

In [ ]:
!git clone https://github.com/pachterlab/kallisto.git
!apt-get install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make

Теперь загружаем и распаковываем файл с индексированным транскриптомом. В общем случае можно сгенерировать его самому, однако для человека он уже есть в открытом доступе. Если потребуется сделать индекс самостоятельно, то следует воспользоваться командой <code>kallisto index</code> (команда работает на <b>транскриптоме</b>, а не геноме &mdash; именно из транскриптома строится референсный граф де Брёйна).

In [ ]:
!wget https://github.com/pachterlab/kallisto-transcriptome-indices/releases/download/ensembl-96/homo_sapiens.tar.gz
!tar -xvzf homo_sapiens.tar.gz && rm homo_sapiens.tar.gz

Теперь всё готово к запуску. Итоговые экспрессии положим в папку <code>expression_data</code>. Для начала запустим на одном образце и посмотрим, что у нас получилось. <b>Эта стадия может занять много времени.</b>

In [ ]:
!mkdir expression_data
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_8 \
    raw_reads/sample_8/ERR4781423_1.fastq.gz raw_reads/sample_8/ERR4781423_2.fastq.gz

Посмотрим, что хранится в файле <code>abundance.tsv</code> в папке <code>expression_data/sample_8/</code>. Для этого выведем строки с 990 по 1000, а также первую (с заголовком).

In [ ]:
!head -n 1 expression_data/sample_8/abundance.tsv | tail -n 10
!head -n 1000 expression_data/sample_8/abundance.tsv | tail -n 10

Видно, что напротив каждого <code>target_id</code> в конце строки стоят колонки <code>est_counts</code> и <code>tpm</code>. Это и есть наши уровни экспрессии. Важно, что, в отличие от результатов работы HTSeq и прочих алгоритмов подсчёта экспрессии, основанных на выравнивании, kallisto <b>оценивает</b> представленность транскриптов, а не физически считает их. Поэтому число ридов на транскрипт <code>est_counts</code> может быть <b>нецелым</b> (позже мы просто округлим его). Проверим, правда ли сумма значений в колонке <code>tpm</code> равна миллиону, а также соответствует ли сумма <code>est_counts</code> числу откартировавшихся прочтений (в логах kallisto срочка <code>processed 29,071,502 reads, 26,150,309 reads pseudoaligned</code>).

In [ ]:
import pandas as pd

sample8_kallisto_output = pd.read_csv("expression_data/sample_8/abundance.tsv",
                                      sep="\t", index_col=0)
tpm_sum = sample8_kallisto_output.tpm.sum()
count_sum = sample8_kallisto_output.est_counts.sum()

print("Counts: {}\nTPM: {}".format(tpm_sum, count_sum))

Всё сходится. Теперь давайте проведём анализ и с оставшимися образцами. <b>Это может занять много времени.</b>

In [ ]:
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_7 \
    raw_reads/sample_7/ERR4781422_1.fastq.gz raw_reads/sample_7/ERR4781422_2.fastq.gz
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_9 \
    raw_reads/sample_9/ERR4781424_1.fastq.gz raw_reads/sample_9/ERR4781424_2.fastq.gz
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_10 \
    raw_reads/sample_10/ERR4781425_1.fastq.gz raw_reads/sample_10/ERR4781425_2.fastq.gz
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_11 \
    raw_reads/sample_11/ERR4781426_1.fastq.gz raw_reads/sample_11/ERR4781426_2.fastq.gz
!./kallisto/build/src/kallisto quant \
    -i homo_sapiens/transcriptome.idx \
    -o expression_data/sample_12 \
    raw_reads/sample_12/ERR4781427_1.fastq.gz raw_reads/sample_12/ERR4781427_2.fastq.gz

Обсчитанные образцы можно загрузить из папки <code>data</code> на GitHub-репозитории курса (файл <code>00. kallisto work.tar.gz</code>). Теперь склеим все обсчитанные экспрессии в одну сводную сводную таблицу.

In [ ]:
expressions_counts = pd.concat(
    [pd.read_csv(
        "expression_data/{}/abundance.tsv".format(expression),
        sep="\t", index_col=0, header=0, usecols=["target_id", expression],
        names=["target_id", "1", "2", expression, "3"]
    ) for expression in ["sample_7", "sample_8", "sample_9", "sample_10", "sample_11", "sample_12"]],
    axis=1, join="inner"
)
expressions_counts.columns = ["sample_7", "sample_8", "sample_9", "sample_10", "sample_11", "sample_12"]
expressions_counts.head()

## Анализ экспрессий

На этом шаге мы будем искать дифференциально экспрессированные между образцами транскрипты.

### rpy2

Из-за того, что анализ дифференциальной экспрессии в основном написан для R, мы будем пользоваться пакетом rpy2, чтобы выполнять код R в Python.

In [ ]:
import rpy2
from rpy2.robjects import r, pandas2ri

%load_ext rpy2.ipython
pandas2ri.activate()

Вот таким образом выглядит ячейка, которая будет выполняться как R-скрипт:

In [ ]:
%%R
a <- c(1,2,3)
a <- a**2
print(a)

Для того, чтобы объекты из окружения R попали в окружение Python (и наоборот), необходимо выполнить следующее:

In [ ]:
a = 20
%R -i a
%R a <- a**2
%R -o a
print(a, type(a))

Если хочется более полно овладеть rpy2, то лучше прочитайте <a href="https://rpy2.github.io/doc/latest/html/index.html" target="_blank">его документацию</a>.

### Анализ дифференциальной экспрессии при помощи DESeq2

Сейчас мы установим <a href="http://bioconductor.org/packages/release/bioc/html/DESeq2.html" target="_blank">DESeq2</a> (<a href="https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8" target="_blank">Love et al, 2014</a>) в наше окружение. Этот пакет нужен для параметрического анализа дифференциальной экспрессии и использует обобщённые линейные модели. На вход он получает матрицу с <b>ненормированными</b> каунтами (не TPM и не RPKM). <b>Установка DESeq2 может занять несколько минут.</b>

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

library("DESeq2")
library(ggplot2)

In [ ]:
%%R
library("DESeq2")
library(ggplot2)

Далее мы перегружаем наш датафрейм из Python в R, чтобы  работать с ним там. Округлим наши экспрессии, чтобы они стали целочисленными (DESeq2 не умеет работать с нецелыми значениями каунтов) &mdash; для этого жёстко обозначим тип данных экспрессии как <code>int32</code>. Также во время портирования в R теряется индекс датафрейма (ID транскриптов), поэтому сохраним его в отдельную переменную.

In [ ]:
expr_matrix = expressions_counts.copy()
expr_matrix = expr_matrix.astype("int32")
expr_matrix.index.name = ""
gene_id = expr_matrix.index.copy()
%R -i expr_matrix
%R -i gene_id

In [ ]:
%%R
rownames(expr_matrix) = make.names(gene_id, unique=TRUE)
head(expr_matrix)

Видим, что в переменной <code>expr_matrix</code> в R содержится то же, что и в переменной <code>expressions_counts</code> в Python.

Теперь необходимо создать <b>аннотацию</b> (переменная <code>colData</code> для каждого семпла). Аннотация представляет из себя ещё один датафрейм. В нём чисто столбцов совпадает с числом столбцов в датафрейме с экспрессиями, а помимо названий образцов там у нас будут различные <b>мета-данные</b>, в том числе и те, которые будут разделять образцы для дальнейшей дифференциальной экспрессии (у нас это будет условием <code>condition</code> с двумя возможными значениями &mdash; WT и Mut).

После из датафреймов <code>expr_matrix</code> и <code>conditions</code> мы сделаем новую переменную <code>dds</code> специального типа, который используется при работе в <code>DESeq2</code>. Также во время создания объекта DESeqDataSet мы указываем условие, по которому будем считать дифференциальную экспрессию. Важно иметь в виду, что модель может оценивать дифференциальную экспрессию в зависимости от сразу нескольких факторов, которые могут быть в том числе и количественные.

In [ ]:
%%R
condition <- c("WT", "WT", "WT", "STK11_del", "STK11_del", "STK11_del")
name <- c("sample_7", "sample_8", "sample_9", "sample_10", "sample_11", "sample_12")
colData <- data.frame(condition, name)
dds <- DESeqDataSetFromMatrix(expr_matrix, colData, design=~condition)
print(dds)

Мы получили специальный объект класса <code>DESeqDataSet</code>. На нём мы выполним функцию <code>DESeq</code>, которая реализует весь пайплайн анализа дифференциальной экспрессии.

In [ ]:
%%R
dds <- DESeq(dds)

Посмотрим, что поменялось в переменной <code>dds</code>.

In [ ]:
%%R
print(dds)

Определим, какие же из транскриптов дифференциально экспрессированы между диким типом и мутантами. Сначала выведем статистику по каждому транскрипту.

In [ ]:
%%R
res <- results(dds)
head(results(dds))

Значения NA у нас возникают у генов, которые никак не экспрессированы ни в одном образце (это нормально для человека). Упорядочим гены по p-value<sub>adj</sub>.

In [ ]:
%%R
res <- res[order(res$padj),]
head(res)

Теперь можно визуализировать разницу между самыми различающимися транскриптами. При помощи функции DESeq2 <code>plotCounts</code> отобразим наглядно, какие уровни экспрессии у самым дифференциально экспрессированных транскриптов в обоих условиях.

In [ ]:
%%R -w 1000
#Задаём размер текста и композицию графика
par(mfrow=c(2,3), cex=1.0, cex.main=1.2, cex.axis=1, cex.lab=0.9)

plotCounts(dds, gene="ENST00000330722.7", intgroup="condition")
plotCounts(dds, gene="ENST00000326873.11", intgroup="condition")
plotCounts(dds, gene="ENST00000307407.8", intgroup="condition")
plotCounts(dds, gene="ENST00000432072.6", intgroup="condition")
plotCounts(dds, gene="ENST00000586243.5", intgroup="condition")
plotCounts(dds, gene="ENST00000456923.5", intgroup="condition")

Построим по нашим результатам VolcanoPlot стандартными методами <code>ggplo2</code>. Вообще, для этого есть удобный пакет <code>EnhancedVolcano</code>, однако через rpy2 он не работает.

In [ ]:
%%R -w 600

#Задаём размер текста и композицию графика
par(mar=c(5,5,5,5), cex=1.0, cex.main=1.7, cex.axis=1, cex.lab=1.5)

#Отрисовываем все точки чёрным цветом
with(res, plot(log2FoldChange, -log10(padj), pch=20, main="Volcano plot", cex=1.0, xlab="log2(FoldChange)", ylab="−log10(p_adj)"))

#Отрисовываем точки, прошедшие порог, красным цветом
with(subset(res, padj<0.001 & abs(log2FoldChange)>3), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5))

#Отрисовываем линии с порогами
abline(v=0, col="black", lty=3, lwd=1.0)
abline(v=-3, col="black", lty=4, lwd=2.0)
abline(v=3, col="black", lty=4, lwd=2.0)
abline(h=3, col="black", lty=4, lwd=2.0)

# Домашнее задание

<b>Дедлайн</b>: 9 ноября, 23:59

<b>Форма отчёта</b>: ссылка на ноутбук в Google Colab или в вашем личном GitHub-репозитории. В ноутбуке должен быть воспроизводимый код, а также ответы на поставленные вопросы.

## Задание 1

Напишите функцию, которая будет принимать на вход матрицу экспрессий в RPKM, а выводить матрицу экспрессий в TPM. Тестовые данные RPKM можете взять в папке <code>data</code> в GitHub-репозитории (файл <code>01. RPKM.tsv</code>). Можно ли так же просто перевести TPM обратно в RPKM?

Все математические выкладки отобразите в ноутбуке (в том числе и вывод формулы перевода).

In [ ]:
def RPKM_to_TPM(RPKM):
  """
  This function return a dataframe with expressions in TPM

  :RPKM: dataframe with expressions in RPKM
  """
  TPM = pd.DataFrame()
  
  # Вставьте ваш код здесь
  
  return TPM

## Задание 2

До этого мы работали на уровне транскриптов. Это логично, ведь различные транскрипты с одного гена могут давать различные по функциям белки, а наша задача &mdash; оценить фенотип образца. Однако мы живём в неидеальном мире, и чаще всего что-то известно о функции какого-то <b>гена</b>, но не каждого транскрипта в частности. В связи с этим возникает задача аггрегации информации от уровня транскриптов (<i>transcript-level</i>) до уровня генов (<i>gene-level</i>).

Наивное решение задачи достаточно просто &mdash; просто складываем каунты на уровне транскриптов до уровней соответствующих им генов. Однако всё не так просто. Представим, что с одного гена продуцируются два различных транскрипта: один имеет длину 10, а другой &mdash; 10 000. А теперь представим себе две крайние ситуации:

1. У нас есть 1 000 молекул первого транскрипта и 1 молекула второго
2. У нас есть 0 молекул первого транскрипта и 2 молекулы второго

Когда мы считаем представленность на уровне транскриптов, мы бы достаточно точно оценили итоговое количество белка в первом и во втором случае. Однако давайте объединим прочтения до уровня генов, что получится в результате? А в результате мы не будем видеть разницу между первым и вторым случаем. Как избежать такой ситуации? Необходимо считать дифференциальную экспрессию на уровне транскриптов, после чего её аггрегировать.

Существуют два пайплайна: первый придуман в лаборатории <a href="https://www.bbe.caltech.edu/people/lior-s-pachter" target="_blank">Лиора Пахтера</a> (это он же придумал kallisto) и называется <a href="https://pachterlab.github.io/sleuth/" target="_blank">sleuth</a> (<a href="https://www.nature.com/articles/nmeth.4324" target="_blank">Pimentel et al, 2017</a>. Помимо стандартного выходного файла <code>abundatnce.tsv</code> он требует файл с бутстреп-репликами <code>abundance.h5</code> (создание этого файла отдельно прописывается при запуске <code>kallisto quant</code>). В чём смысл бутстреп-реплик можно прочитать <a href="https://liorpachter.wordpress.com/2015/05/10/near-optimal-rna-seq-quantification-with-kallisto/" target="_blank">здесь</a> (если кратко, то они помогают лучше оценить дисперсию для параметрической модели).

Второй подход основан на библиотеке <a href="https://bioconductor.org/packages/release/bioc/html/tximport.html" target="_blank">tximport</a> (<a href="https://f1000research.com/articles/4-1521/v2" target="_blank">Soneson et al, 2016</a>) для R. В ходе процессинга при помощи tximport учитывается неравномерное распределение длин изоформ гена &mdash; эта поправка напрямую учитывается в DESeq2, анализ которого теперь будет вызываться функцией <code>DESeqDataSetFromTximport()</code>.

Ваша задача &mdash; выполнить анализ дифференциальной экспресси на уровне <b>генов</b> с использованием tximport (<a href="https://bioconductor.org/packages/release/bioc/vignettes/tximport/inst/doc/tximport.html" target="_blank">инструкцию можно прочитать тут</a>).

In [ ]:
# Установка tximport
BiocManager::install("tximport")
library(tximport)

In [ ]:
tx2gene = pd.read_table("/content/homo_sapiens/transcripts_to_genes.txt", sep="\t", names = ["1", "2", "3"]) 
del tx2gene["3"] 

In [ ]:
%R -i tx2gene 

In [ ]:
%%R 

files <- c("expression_data/abundance_sample7.tsv", "expression_data/abundance_sample8.tsv", "expression_data/abundance_sample9.tsv", "expression_data/abundance_sample10.tsv", "expression_data/abundance_sample11.tsv", "expression_data/abundance_sample12.tsv") 
names(files) <- paste0("sample", 7:12) 
files 

In [ ]:
# Подготовка при помощи tximport 
%%R 
txi.kallisto.tsv <- tximport(files, type = "kallisto", tx2gene = tx2gene, ignoreAfterBar = TRUE) 
head(txi.kallisto.tsv$counts) 

In [ ]:
# Обработка при помощи DESeq2 
%%R 
condition <- c("WT", "WT", "WT", "STK11_del", "STK11_del", "STK11_del") 
name <- c("sample_7", "sample_8", "sample_9", "sample_10", "sample_11", "sample_12") 
colData <- data.frame(condition, name) 
rownames(colData) <- colnames(txi.kallisto.tsv$counts) 

dds <- DESeqDataSetFromTximport(txi.kallisto.tsv, colData, ~condition) 
print(dds) 

In [ ]:
%%R 
dds <- DESeq(dds) 

In [ ]:
%%R 
res <- results(dds) 
head(results(dds)) 

In [ ]:
%%R 
res <- res[order(res$padj),] 
head(res) 

In [ ]:
%%R -w 1000 
#Задаём размер текста и композицию графика 
par(mfrow=c(2,3), cex=1.0, cex.main=1.2, cex.axis=1, cex.lab=0.9) 

plotCounts(dds, gene="ENSG00000205420.11", intgroup="condition") 
plotCounts(dds, gene="ENSG00000169429.11", intgroup="condition") 
plotCounts(dds, gene="ENSG00000057149.16", intgroup="condition") 
plotCounts(dds, gene="ENSG00000115414.19", intgroup="condition") 
plotCounts(dds, gene="ENSG00000081181.8", intgroup="condition") 
plotCounts(dds, gene="ENSG00000027869.11", intgroup="condition") 

In [ ]:
%%R -w 600 

#Задаём размер текста и композицию графика 
par(mar=c(5,5,5,5), cex=1.0, cex.main=1.7, cex.axis=1, cex.lab=1.5) 

#Отрисовываем все точки чёрным цветом 
with(res, plot(log2FoldChange, -log10(padj), pch=20, main="Volcano plot", cex=1.0, xlab="log2(FoldChange)", ylab="−log10(p_adj)")) 

#Отрисовываем точки, прошедшие порог, красным цветом 
with(subset(res, padj<0.001 & abs(log2FoldChange)>3), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5)) 

#Отрисовываем линии с порогами 
abline(v=0, col="black", lty=3, lwd=1.0) 
abline(v=-3, col="black", lty=4, lwd=2.0) 
abline(v=3, col="black", lty=4, lwd=2.0) 
abline(h=3, col="black", lty=4, lwd=2.0)

Если интересно подробнее узнать про разницу между дифференциальной экспрессией на генах и на транскриптах, советую посмотреть два видео Лиора Пахтера: <a href="https://www.youtube.com/watch?v=T9oqQOkW7V8" target="_blank">первое</a> и <a href="https://www.youtube.com/watch?v=ucPBBTjH5EE" target="_blank">второе</a>.

## Задание 3

Из GitHub-репозитория (папка <code>data</code>) загрузите архив <code>03. kallisto hometask.tar.gz</code> для шести дополнительных образцов, которые были в исследовании. Выполните анализ, аналогичный тому, что мы делали на занятии: посмотрите на дифференциально экспрессированные гены для (1) просто нокаутов по <i>KEAP1</i> и для (2) нокаутов по обоим генам. <b>Дифференциальную экспрессию рассчитывайте на уровне генов, а не транскриптов.</b>

Являются ли дифференциально экспрессированные гены во втором случае объединением дифференциально экспрессированных генов по нокаутом <i>KEAP1</i> и <i>STK11</i>? Почему?

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

library("DESeq2")
library(ggplot2)

In [ ]:
%%R
library("DESeq2")
library(ggplot2)

In [ ]:
expr_matrix = expressions_counts.copy()
expr_matrix = expr_matrix.astype("int32")
expr_matrix.index.name = ""
gene_id = expr_matrix.index.copy()
%R -i expr_matrix
%R -i gene_id

In [ ]:
%%R
rownames(expr_matrix) = make.names(gene_id, unique=TRUE)
head(expr_matrix)

In [ ]:
%%R
condition <- c("WT", "WT", "WT", "STK11_del", "STK11_del", "STK11_del")
name <- c("sample_13", "sample_14", "sample_15", "sample_16", "sample_17", "sample_18")
colData <- data.frame(condition, name)
dds <- DESeqDataSetFromMatrix(expr_matrix, colData, design=~condition)
print(dds)

In [ ]:
%%R
dds <- DESeq(dds)

In [ ]:
%%R
print(dds)

In [ ]:
%%R
res <- results(dds)
head(results(dds))

In [ ]:
%%R
res <- res[order(res$padj),]
head(res)

In [ ]:
%%R -w 1000
#Задаём размер текста и композицию графика
par(mfrow=c(2,3), cex=1.0, cex.main=1.2, cex.axis=1, cex.lab=0.9)

plotCounts(dds, gene="ENST00000631435.1", intgroup="condition")
plotCounts(dds, gene="ENST00000434970.2", intgroup="condition")
plotCounts(dds, gene="ENST00000448914.1", intgroup="condition")
plotCounts(dds, gene="ENST00000415118.1", intgroup="condition")
plotCounts(dds, gene="ENST00000632684.1", intgroup="condition")
plotCounts(dds, gene="ENST00000604446.1", intgroup="condition")

In [ ]:
%%R -w 600

#Задаём размер текста и композицию графика
par(mar=c(5,5,5,5), cex=1.0, cex.main=1.7, cex.axis=1, cex.lab=1.5)

#Отрисовываем все точки чёрным цветом
with(res, plot(log2FoldChange, -log10(padj), pch=20, main="Volcano plot", cex=1.0, xlab="log2(FoldChange)", ylab="−log10(p_adj)"))

#Отрисовываем точки, прошедшие порог, красным цветом
with(subset(res, padj<0.001 & abs(log2FoldChange)>3), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5))

#Отрисовываем линии с порогами
abline(v=0, col="black", lty=3, lwd=1.0)
abline(v=-3, col="black", lty=4, lwd=2.0)
abline(v=3, col="black", lty=4, lwd=2.0)
abline(h=3, col="black", lty=4, lwd=2.0)

## Задание 4<sup>*</sup>

Снизьте размерность транскрипционных профилей образцов при помощи метода клавных компонент. Постройте график по первым двум компонентам из PCA. Как именно это делается можно прочитать <a href="https://www.machinelearningmastery.ru/pca-using-python-scikit-learn-e653f8989e60/" target="_blank">тут</a> (в R это делается одной функцией <code>prcomp</code>). Объясните получившиеся результаты.

In [ ]:
# Вставьте ваш код здесь